# Template Attack

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import random

import lascar
import numpy as np
import plotly.graph_objects as pgo
from cwtoolbox import CaptureDevice

In [ ]:
capture_device = CaptureDevice.create("CWLITEXMEGA")
capture_device.compile(code="volatile uint8_t i = input[0];")
capture_device.flash()

In [ ]:
datas = [
    capture_device.capture(number_of_traces=1000, input=lambda _: value)
    for value in (b"\x00", b"\xf0", b"\xff")
]

In [ ]:
fig = pgo.Figure()
for trace in data[:20]:
    fig.add_trace(pgo.Scatter(y=trace["trace"]))
fig.show()

In [ ]:
poi = 25
fig = pgo.Figure()
for data in datas:
    hist = np.histogram(data["trace"][:, poi], bins=8, density=True)
    fig.add_trace(pgo.Bar(y=hist[0], x=hist[1], width=0.001))
fig.show()


## Find Points of Interest

In [ ]:
import itertools

data = capture_device.capture(number_of_traces=10000, input=lambda _: random.randbytes(1))

def group_traces_by_hw(data):
    return {
        hw: np.array(
            [
                trace["trace"]
                for trace in data
                if lascar.hamming(trace["input"][0]) == hw
            ]
        )
        for hw in range(9)
    }


def grouped_diffs(data_grouped):
    means = {
        hw: np.nan_to_num(np.nanmean(data_grouped[hw], axis=0))
        for hw in range(9)
    }
    return sum(
        (meani - meanj) ** 2
        for meani, meanj in itertools.product(means.values(), repeat=2)
    )

# Calculate the diff "where the traces differ at most"
diffs = grouped_diffs(group_traces_by_hw(data))
pois = np.argsort(diffs)[-5:]

fig = pgo.Figure()
fig.add_trace(pgo.Scatter(y=diffs))
fig.show()

pois

## Generate Templates

In [ ]:
import sklearn.discriminant_analysis

def selection_function(value):
    return lascar.hamming(value["input"][0])

classifier_qda = sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis(store_covariance=True)
classifier_profile_engine_qda = lascar.ProfileEngine(
    classifier=classifier_qda,
    partition_function=selection_function,
    partition_range=range(9),
)
trace = lascar.TraceBatchContainer(data["trace"], data)
trace.leakage_section = pois
lascar.Session(
    container=trace,
    engine=classifier_profile_engine_qda
).run()

## Attack using Templates

In [ ]:
value = b"\x01"
data_attack = capture_device.capture(number_of_traces=1, input=lambda _: value)

In [ ]:
def selection_function(value, guess):
    return lascar.hamming(guess)

classifier_match_engine_qda = lascar.MatchEngine(
    classifier=classifier_qda,
    selection_function=selection_function,
    guess_range=range(256),
)
trace = lascar.TraceBatchContainer(data_attack["trace"], data_attack)
trace.leakage_section = pois
session = lascar.Session(
    trace,
    engine=classifier_match_engine_qda,
    output_method=lascar.TableOutputMethod(classifier_match_engine_qda),
).run()